In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
from torchvision.transforms import Compose, ToTensor
from torchsummary import summary
import pickle
import pandas as pd
import plotly.express as px
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import accuracy_metrics

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [5]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import CEWCV1
from base_code.plugins import WeightStoragePlugin

# Dataset and definitions

## Preprocessing definitions

In [6]:
train_transform = Compose(
    [
        ToTensor(),
    ]
)

test_transform = Compose(
    [
        ToTensor(),
    ]
)

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [7]:
# mnist_train = MNIST(DATASETS_PATH, train=True, download=True, transform=train_transform)
# mnist_test = MNIST(DATASETS_PATH, train=False, download=True, transform=test_transform)

scenario = PermutedMNIST(10, seed=1234, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [8]:
# scenario = nc_benchmark(
#     mnist_train, mnist_test,
#     n_experiences=len(mnist_train.classes), shuffle=True, seed=1234, task_labels=False
# )

train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [15]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True), loggers=[InteractiveLogger()]
)

## Plugin defitinitions

In [16]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [18]:
# model = MLP(n_classes=scenario.n_classes, n_channels=1, width=28, height=28)
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss()
strategy = CEWCV1(
    model,
    optimizer,
    criterion,
    ewc_lambda_l1=1.0,
    ewc_lambda_l2=1.0,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [19]:
results = list()

for experience in tqdm(train_stream[:1]):
    strategy.train(experience)
    metrics = strategy.eval(test_stream)
    results.append(metrics)

  0%|          | 0/1 [00:00<?, ?it/s]

-- >> Start of training phase << --
Used params initialized to zero tensor([False, False, False,  ..., False, False, False])
100%|██████████| 469/469 [00:04<00:00, 112.60it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 114.92it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:04<00:00, 112.93it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:04<00:00, 114.37it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:04<00:00, 114.16it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 143.31it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7680
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 143.30it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1613
-- Starting eval on exp

100%|██████████| 1/1 [00:29<00:00, 29.22s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1858
-- >> End of eval phase << --
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.2223


In [20]:
strategy.get_store_loss()

defaultdict(list,
            {'first_component': [4.615281105041504,
              4.632040977478027,
              4.636293888092041,
              4.641708850860596,
              4.632201194763184,
              4.606926918029785,
              4.635421276092529,
              4.59839391708374,
              4.61265754699707,
              4.617992877960205,
              4.632148265838623,
              4.627546787261963,
              4.624602794647217,
              4.6250834465026855,
              4.652464389801025,
              4.634077072143555,
              4.588904857635498,
              4.606813430786133,
              4.596785545349121,
              4.620111465454102,
              4.616281032562256,
              4.614748001098633,
              4.632349967956543,
              4.596991539001465,
              4.605758190155029,
              4.588881015777588,
              4.629431247711182,
              4.602360725402832,
              4.599004745483398,
       

In [12]:
accuracies: dict[int, list[float]] = dict()

for i in range(10):
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp00{i}"
    ][1][-10:]

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1][-10:]

In [13]:
acc_df = pd.DataFrame(accuracies)
acc_df.index = range(10)

In [17]:
fig = px.line(acc_df, x=acc_df.index, y=acc_df.columns, range_y=[0, 1])
fig.show()

# Store metrics

In [15]:
pickle.dump(accuracies, open(SAVED_METRICS_PATH / "cewc_v1.pkl", "wb"))